In [2]:
from google.colab import drive
drive.mount('/content/drive')

PATH_OF_LSTM="[YOUR_PATH]/main/replica_models/LSTM"
PATH_OF_LSTM="/content/drive/MyDrive/Machine Learning and Deep Learning/Project/bert replica evaluation/bert-gen-master/bert-gen-master/main/replica_models/LSTM"
TRAIN_DATA_PATH = PATH_OF_LSTM +"/wiki.train.tokens"
PATH_OF_DATA="[YOUR_PATH]/main/data"
PATH_OF_DATA="/content/drive/MyDrive/Machine Learning and Deep Learning/Project/bert replica evaluation/bert-gen-master/bert-gen-master/main/data"

import sys
if PATH_OF_LSTM not in sys.path:#if append the same path multiple time it generate error
  sys.path.append(PATH_OF_LSTM) #Adding the path of LSTM in order to import 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from LSTM_Model import Model, train, last_word_logits, generate_sentence
from LSTM_Dataset import Dataset
import torch
import math
import numpy as np
import time
import os



# Load model and tokenizer
Dataset class implement the tokenizer.

Note: the funciton used last_word_logits work only with cuda, please use this as device

In [6]:
dataset = Dataset(TRAIN_DATA_PATH)     # read the training dataset and makes little data prep
model = Model(dataset)  # create and load the prepare the LSTM Model


#load the pretrained model

model.load_state_dict(torch.load(os.path.join(PATH_OF_LSTM,"filename.pth")))
#model.cuda()
model.eval()
for p in model.parameters():
  print(len(p))

33279
160
160
160
160
33279
33279


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
'''
    TRAINING PHASE
    DON'T RUN THIS CELL
'''

#train(dataset, model)
#torch.save(model.state_dict(), os.path.join(PATH,"filename.pth"))

"\n    TRAINING PHASE\n    DON'T RUN THIS CELL\n"

#Utility function

In [ ]:
def get_init_text(batch_size = 1,seed_text="" ):

    words=seed_text.split(" ")
    x = [[dataset.word_to_index[w] for w in words] for _ in range(0,batch_size)]
    return torch.tensor(x)

In [ ]:
def write_sents(out_file, sents, should_detokenize=False):
  '''Write a list of lists of word to a file
      Use should_detokenize to detokenize sents
  '''
  with open(out_file, "w") as out_fh:
  
      for sent in sents:
          text=""
          for tok in sent:
            if tok[0]=="\n":
              text=text+" "+"\\n"
              continue
            text=text+" "+tok[0]
          out_fh.write("%s\n" % text)

#Generation functions

In [ ]:
def generate_step(logits, temperature=None, top_k=0, sample=False, return_list=True):
    """ Generate a word from from out[gen_idx]
    
    args:
        - out (torch.Tensor): tensor of logits of size batch_size x seq_len x vocab_size
        - gen_idx (int): location for which to generate for
        - top_k (int): if >0, only sample from the top k most probable words
        - sample (Bool): if True, sample from full distribution. Overridden by top_k 
    """
    #logits = out[:, gen_idx] #get the logits for the selected index (the distribution of the words of the vocabulary for the specified index)
    if temperature is not None:
        logits = logits / temperature#To smooth the distribution (see generate function)
    if top_k > 0: #To sample from top_k word if top_k>0 (Note with top_k=1 is the same of the argmax i.e. get the prediction of Bert)
        kth_vals, kth_idx = torch.topk(logits,top_k)
        dist = torch.distributions.categorical.Categorical(logits=kth_vals)
        idx = kth_idx.gather(dim=1, index=dist.sample().unsqueeze(-1)).squeeze(-1)
    elif sample: #To sample from all distribution if top_k=0
        dist = torch.distributions.categorical.Categorical(logits=logits)
        idx = dist.sample().squeeze(-1)
    else: #To avoid sample and get the prediction of Bert if top_k = 0 and sample=False
        idx = torch.argmax(logits, dim=-1)
    return idx.tolist() if return_list else idx

def parallel_sequential_generation(seed_text,batch_size=1, top_k=0, temperature=None, max_iter=300, burnin=200,
                                   cuda=False, print_every=10, verbose=True):
    """ Generate for one random position at a timestep
    
    args:
        - seed_text: write a sentence as start point or use "" to start from only mask
        - batch_size: specify number of starting sentence
        - max_len: specify the number of mask to add in the starting sentence
        - top_k: to sample from top_k word of the distribution
        - temperature: to smooth distribution
        - max_iter: number of timestep
        - burnin: during burn-in period, sample from full distribution; afterwards take argmax
        - cuda: to use GPU
        - print_every: if verbose to print every x timestep
    """
 
    max_len=len(seed_text.split(" "))
    batch = get_init_text(batch_size,seed_text=seed_text)
 
 
    for ii in range(max_iter):
        kk = np.random.randint(max_len-1)+1  #select a random index from 1 to sentence len -1of the mask

        input_sentence=[]
        for jj in range(batch_size):
            input_sentence.append(batch[jj][0:kk].cpu().numpy())
          
        #print(kk," ",input_sentence)
        inp = torch.tensor(input_sentence).cuda() if cuda else torch.tensor(batch) #send to cuda if avalaible
        out = last_word_logits(dataset,model,inp,cuda) #get the logits for each sentence in the batch
        topk = top_k if (ii >= burnin) else 0 #burnin period if top_k = 0 and sample =False the function sample from all distribution
        idxs = generate_step(out,top_k=topk, temperature=temperature, sample=(ii < burnin)) #generate the word 
        for jj in range(batch_size):
            batch[jj][kk] = idxs[jj] #put the generated word inside the sentence
            
        if verbose and np.mod(ii+1, print_every) == 0:
            for_print = [[[dataset.index_to_word[int(idx)] ] for idx in batch[jj]] for jj in range(batch_size)]
            #for_print = for_print[:seed_len+kk+1] + ['(*)'] + for_print[seed_len+kk+1:]
            print("index: ",kk,"iter", ii+1)
            print(for_print)
            
    return [[[dataset.index_to_word[int(idx)] ] for idx in batch[jj]] for jj in range(batch_size)]

def generate(n_samples, seed_text="", batch_size=10, max_len=25, 
             sample=True, top_k=100, temperature=1.0, burnin=200, max_iter=500,
             cuda=False, print_every=1):
    # main generation function to 
    

    sentences = []
    n_batches = math.ceil(n_samples / batch_size) #number of batches
    start_time = time.time()
    for batch_n in range(n_batches):
        batch = parallel_sequential_generation(seed_text,batch_size=batch_size, top_k=top_k,
                                               temperature=temperature, burnin=burnin, max_iter=max_iter, 
                                               cuda=cuda, verbose=False)
        
   
        if (batch_n + 1) % print_every == 0:
            print("Finished batch %d in %.3fs" % (batch_n + 1, time.time() - start_time))
            start_time = time.time()
            print(batch)
        
        sentences += batch
    return sentences

# Generate sentences

Generate and store sentence

In [ ]:
n_samples = 1000#1000
batch_size = 50#50

top_k = 100
temperature = 0.7

#We used 80 iteration tha authors instead 500, but the time to do 500 iterarion is to much with our lstm
burnin = 40
sample = True
max_iter = 80

seed = "As an armed Gallian force invading the Empire just following the two nations cease fire would certainly wreck their newfound peace , Kurt decides to once again make his squad the Nameless , asking Crowe to list himself and all"

generation=generate(n_samples, seed_text=seed, batch_size=batch_size,
                          sample=sample, top_k=top_k, temperature=temperature, burnin=burnin, max_iter=max_iter,
                          cuda=True)

In [ ]:
out_file = PATH_OF_DATA+"/LSTM-len%d-burnin%d-topk%d-temp%.3f-generation.txt" % (40, burnin, top_k, temperature)
write_sents(out_file, generation)

##Generate sentence with checkpoint

In [ ]:
n_samples = 50#1000
batch_size = 25#50

top_k = 100
temperature = 0.7

#We used 80 iteration tha authors instead 500, but the time to do 500 iterarion is to much with our lstm
burnin = 40
sample = True
max_iter = 80

seed = "As an armed Gallian force invading the Empire just following the two nations cease fire would certainly wreck their newfound peace , Kurt decides to once again make his squad the Nameless , asking Crowe to list himself and all"
for i in range(14,16):
  generation=generate(n_samples, seed_text=seed, batch_size=batch_size,
                            sample=sample, top_k=top_k, temperature=temperature, burnin=burnin, max_iter=max_iter,
                            cuda=False)

  out_file = PATH_OF_DATA+"/LSTM-len%d-burnin%d-topk%d-temp%.3f-generation50-%d.txt" % (40, burnin, top_k, temperature,i)
  write_sents(out_file, generation)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Finished batch 1 in 234.381s
[[['As'], ['native'], ['a'], ['.'], ['force'], ['and'], ['the'], [','], ['.'], ['<unk>'], ['.'], ['the'], ['the'], ['to'], ['to'], ['the'], ['the'], ['wreck'], ['\n'], ['newfound'], ['to'], ['.'], ['for'], ['decides'], ['in'], ['once'], ['.'], [','], ['a'], ['@-@'], ['affected'], ['in'], ['may'], [','], [','], ['by'], ['boxes'], ['.'], ['in'], ['with']], [['As'], ['the'], ['estimated'], ['may'], ['force'], ['of'], ['the'], ['<unk>'], ['<unk>'], ['bird'], ['the'], ['.'], [','], ['<unk>'], ['<unk>'], ['<unk>'], ['and'], ['wreck'], ['was'], ['newfound'], ['to'], ['to'], [','], ['decides'], ['estimated'], ['once'], ['bird'], ['of'], ['.'], ['in'], ['.'], ['is'], ['the'], ['in'], [','], ['eat'], ['to'], ['was'], ['in'], ['is']], [['As'], ['sites'], ['a'], ['in'], ['force'], ['to'], ['the'], [','], [','], ['in'], ['remove'], ['<unk>'], ['the'], ['the'], ['%'], ['to'], ['the'], ['wreck'], ['of'], ['newfound'], ['feed'], ['.'], ['<unk>'], ['decides'], ['on'], ['onc